<a href="https://colab.research.google.com/github/YoungPrzemoDev/AI-Projects/blob/main/BUILDING_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Attention is all you need
#GPT - genratively pre-trained Transformer
#Transformer - neural net that does heavy lifing under the hood

In [ ]:
#TOKENIZACJA - zmiana tekstu jako string na intigery

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-03-10 13:44:47--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2024-03-10 13:44:47 (32.5 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [ ]:
#Hyperparameters
batch_size=32 # ile niezaleznych sekwencji bedzie procesowanych jednocześnie
block_size = 8 # dlugosc contekstu do predykcji (context lenght)
max_iters = 3000
eval_interval=300
learning_rate=1e-3
eval_iters=200
n_embd=32

In [ ]:
#With - praca z zasobami, automatycznie otwiera i zamyka zasób
# 'r' ze read
with open ('input.txt','r',encoding='utf-8') as f:
  text = f.read()

In [ ]:
print("Total length in characters: ", len(text))

Total length in characters:  1115394


In [ ]:
#Potrzebujemy posortowane znaki wiec uzywamy sorted ale funkcja sorted wymaga sekwencji np listy wiec dlatego jest
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
#Tokenizer

# klucz : wartosc for index, wartosc in enumerate(iterowalny_obiekt)
# klucz : wartosc - to jest jakby nasz output, to otrzymujey po petli for, czyli akurat tutaj dictionary w pythonie ale to taka lista klucz wartosc
stoi={ s:i for i,s in enumerate(chars) } # klameerki są do tworeznie słowników
itos = { i:s for i,s in enumerate(chars)}
encode = lambda s : [stoi[c] for c in s ]
# List comprehension [itos[i] for i in l] przekształci [1, 2, 3] na ['a', 'b', 'c'].
# A gdy dodamy ''.join() połączy te znaki w jeden ciąg znaków "abc" bez żadnego separatora,
decode = lambda l : ''.join([itos[i] for i in l])

print(encode("Bunkrow nie ma, ale tez jest zajebiscie"))
print(decode(encode("Bunkrow nie ma, ale tez jest zajebiscie")))

[14, 59, 52, 49, 56, 53, 61, 1, 52, 47, 43, 1, 51, 39, 6, 1, 39, 50, 43, 1, 58, 43, 64, 1, 48, 43, 57, 58, 1, 64, 39, 48, 43, 40, 47, 57, 41, 47, 43]
Bunkrow nie ma, ale tez jest zajebiscie


In [ ]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:1000])

In [ ]:
#Data set
n = int(0.9 * len(data)) # 90 prcoent
train_data = data[:n] #do 90 prcoent
val_data = data[n:] #po 90 procentach#

In [ ]:
#Podczas uczenia nie wkladamy calego tekstu tylko male kawałki bo computation za duzo przy calym
train_data[:block_size + 1]

#tensor([18, 47, 56, 57, 58,  1, 15, 47, 58]) - każdy token patrzy na wszystkie poprzednie tokeny aby lepiej zrozumieć jak powinien wyglądać nastepny token.
#W tradycyjnych modelach sekwencyjnych, jak RNN, przewidywanie kolejnych tokenów odbywa się sekwencyjnie.
#W modelu Transformera jednak wszystkie pozycje są przetwarzane równocześnie (w jednym przejściu przez model),
#co jest dużo bardziej efektywne obliczeniowo ale nie robimy tego tylko po to. Równierz po to aby nasz transformer przywykł
#do widzenia kontextu od najmniejszego aż do block size, że np wie po 1 znaku co ma byc dalej jako i po 5 czy 8 itp.
#Potem zaczynamy obcinac i zaczynac kolejny block size bo przecież bedzie wiedział tylko do naszego block size

In [ ]:
#Przykład tego jak każdy token patrzy na wszystkie poprzednie tokeny:
x = train_data[:block_size]
y = train_data[:block_size+1]
for i in range(block_size):
  context = x[:i+1] # do 1,2,3... indexu itp.
  pred = y[i+1]
  print(context, " --> ", pred )

In [ ]:
len(data)

1115394

In [ ]:
#Batch
torch.manual_seed(1337)

def getBatch(split):
  if(split=='train'):
    data = train_data
  else:
    data = val_data
  #torch.randint(max_val, shape) -  tworzy tensor z random integers generowane uniformly between
  #0 and the first argument a 2 argument to shape
  #len(data) - zwraca rozmiar 1 wymiaru odpowiadającego za ilosc przykladow
  #len(data)-block_size - Na przykład, jeśli data ma 1000 elementów, a block_size wynosi 100,
  #to maksymalny indeks startowy, który możesz wybrać, to 900.
  #W ten sposób ostatni wybrany blok danych będzie zawierał elementy od 900 do 999 włącznie, co zapewnia,
  #że nie próbujesz sięgać po dane poza dostępnym zakresem bo randomo wybieramy z datasetu .
  ix = torch.randint(len(data) - block_size, (batch_size,)) # dostajemy indexy tyle co batch_size ktore sa poczatkiem dla sekwencji danych block_size
  #print(ix)

  #stackujesz sobie jedno pod drugim , w sensie ze dokladasz wiersz
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y

xb,yb = getBatch('train')
print('inputs: ' , xb.shape)
print(xb)
print('targets: ' , yb.shape)
print(yb)

print('------')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b,t]
    print(f"when input is{context.tolist()} the target: {target}")

In [ ]:
class Block(nn.Model):
  def __init__(self,n_embd,n_head):
    super().__init__()
    head_size = n_embd// n_head
    #Communication
    #self attention head. skoro 4 heady to C dzielimy na 4 bo kazdy bedzie pracowal na tych samych danych ale w innych wymiarach
    self.sa = MultiheadAttention(n_head,head_size)
    #Computation
    self.ffwd = FeedForward(n_embd)

    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self,x):
    #Communication
    #zanim przejdzie do sa i fe musimy znormalizwoac x
    x = x + self.sa(self.ln1(x)) # x = x + self.sa(x) - to jest potrzebne do optymalizacji bardzo glebokich sieci i nazywa sie residual connections
    #Computation
    x = x + self.ffwd(self.ln2(x)) # x = x + self.ffwd(x) - to jest potrzebne do optymalizacji bardzo glebokich sieci i nazywa sie residual connections
    return x

NameError: name 'nn' is not defined

In [ ]:
#Jesli ammy bardzo gleboka siec jak tu optymalizacja ubolewa.
#Mozemy zrobic 2 rzeczy ktore pomoga

In [ ]:
import torch
import torch.nn as nn # importuje moudly i klasy
from torch.nn import functional as F # importuje gotowe funkjce
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module): # dziedzicze po klasie Module

  def __init__(self):
    super().__init__() # konstruktor Module
    #Tworzymy tablice embedingow, Jest to tabela embeddingów dla tokenów
    self.token_embedding_table = nn.Embedding(vocab_size,n_embd)

    #Jest to tabela embeddingów dla pozycji tokenów w sekwencji,
    #Embedding pozycyjny ma na celu dostarczenie informacji o położeniu tokenu w sekwencji,
    #co jest kluczowe dla modeli językowych, aby rozumieć kontekst i zależności między tokenami
    #w zależności od ich pozycji.
    #Przykładowo, słowo "bank" może mieć zupełnie inne znaczenie w zależności od tego, czy znajduje się
    #po słowie "rzeczny" czy "pieniężny",
    self.position_embedding_table = nn.Embedding(block_size,n_embd)
    self.blocks = nn.Sequential(
        Block(n_embd, h_head=4),
        Block(n_embd, h_head=4),
        Block(n_embd, h_head=4),
        nn.layerNorm(n_embd), # na koncu transformera tez powinna byc normalizacja i tuz przed finalna liniowa ktroa przekształca potem w vocab_size
    )
    self.lm_head = nn.Linear(n_embd,vocab_size)


  def forward(self, idx, targets=None): # None by default bo przy generowaniu nie potrzebne
    B, T = idx.shape #idx jest 2 wymiarowy

    #Przy przekazaniu tensora idx do modelu, każda liczba całkowita (indeks) w tym tensorze
    #odnosi się do wiersza w tabeli embeddingów i "wybiera" (ang. pluck out) z niej odpowiedni wiersz (embedding).
    #Na przykład indeks 24 spowoduje wybranie 24-go wiersza z tabeli embeddingów.
    token_emb = self.token_embedding_table(idx) # (B,T,C) batch, time , channel
    #pozwala modelowi na uwzględnienie informacji o kolejności tokenów w sekwencji.
    #arrange - Generuje embeddingi pozycyjne na podstawie sekwencji indeksów od 0 do T-1
    pos_emb = self.position_embedding_table(torch.arange(T)) #(T,C)
    x = token_emb + pos_emb # (B,T,C) bo dzial ten broadcasting
    x = self.sa_heads(x)
    x = self.ffwd(x)
    logits = self.lm_head(x) #(B,T,vocab_size), to jest fully conected linear layer

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits,targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    #idx jest w formie (B,T)
    for _ in range(max_new_tokens):

      #!!!!!!!!spradz jeszcze to uzywamy teraz jeszcze positiona embedings wiec musimy miec pewnosc ze otrzymamy tylko block_size anie wiecej
      idx_cond = idx[:,-block_size:]

      #slef(idx) uruchomienie forward i wez predykcje shape- (B,T,C)
      logits, loss = self(idx_cond) #Dostaje tylko logity a loss ignorowane bo nie dostaje targeetów z ktorymi moze sobie porownac
      logits = logits[:, -1, :]# becomes (B,C), interesuje nas ostatni krok czasowy bo chcemy predykcje zawsze dla ostatniego
      #daj softmax zeby dostac prawdopodobienstwo
      probs = F.softmax(logits, dim=1) # (B,C)
      #sample from distribution
      idx_next = torch.multinomial(probs,num_samples=1) # (B,1)
      #concaconete to the running sequence
      idx = torch.cat((idx, idx_next), dim=1) # (B,T+1)
    return idx


m = BigramLanguageModel()
logits, loss = m(xb,yb) # podajemy targets wiec wykona sie else w forward
print(logits.shape)
print(loss)
# tensor 1 na 1 i ma 0 czyli początek po prpstu 0 to new line character w naszym  slowniku
# [0].tolits odnosi sie do calej sekwencji wygenerowanej
print(decode(m.generate(torch.zeros((1,1), dtype = torch.long),100)[0].tolist()))

torch.Size([512, 65])
tensor(4.4519, grad_fn=<NllLossBackward0>)

awhbAGKFj;TjIyV,
-vbyWjPEZQliW!XfWAwWafJ'GfOuv$ARqeemreKRRVemX!F$Xw!
XvoyAIhdnCM3S;qR&AyyRoXoIjGSRpw


In [ ]:
#Optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:

for steps in range(1000):
  xb,yb = getBatch('train')

  #evaluate the loss
  logits, loss = m(xb,yb)
  optimizer.zero_grad(set_to_none=True) #This clears old gradients from the last step
  loss.backward()
  optimizer.step()

print(loss.item())

2.506159543991089


In [ ]:
print(decode(m.generate(torch.zeros((1,1), dtype = torch.long),400)[0].tolist()))

The mathematical trick in self attention

In [ ]:
#self-attention is beiung used inside a Transformer i jest jak by sercem wydajności

In [ ]:
torch.manual_seed(1337)
B,T,C = 4,8,2
#tensor o rozmairze podanym i randomowe liczby z rozkladu normalnego
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
 #Version 1
 xbow = torch.zeros(B,T,C)
 for b in range(B):
  for t in range(T):
    #Jak wybieram odpowiedni batch czyli B to redukuje wymiarowość,
    xprev = x[b, :t+1] # (t,C)
    #torch.mean(xprev,0) - zlicza nam mean xprev po pierwsszym wymiarze i przypisuje ja do tokenu znajdujacego sie na podanych indexach
    xbow[b,t] = torch.mean(xprev,0)


In [ ]:
#Version 2
wei = torch.tril(torch.ones(T,T)) # daje 1 na przekatnej i pod nia a powyzej zera, potrzebne do oblcizenia mean
wei = wei / wei.sum(1,keepdim=True) #(8,8), x = (4,8,2)
# wei zostal rozciagniety o wymiar i ma teraz (1,8,8) zamiast (8,8) bo dziala mechanizm broadcastingu
# dzial tak ze patrzy od konca na wymiary jest 8 i 2 potem 8 i 8 a 4 nie ma z czym poronwac wiec rozciaga o wymiar
xbow2 = wei @ x # (1,8,8) @ (4,8,2) dziala bo nie patrzymy na 8 i 4  tylko 8 i 8 oraz 8 i 2 a piewszy eymiar jest rozciagany na 4 razy
xbow2

In [ ]:
#Version 3 using softmax
tril = torch.tril(torch.ones(T,T))
#zaczynaja sie do 0 wszędzie i myśl o nich jak o sile interakcji miedzy tokenami
#one sa tylko na poczatku zero a potem zalezne od danych, tokeny beda patrzec po sobie i sprawdzac ktory ich bardziej interesuje
wei = torch.zeros(T,T)
#Tutaj mowimy ze tokeny z przeszlosci nie moga odwolywac sie do przyszlosci
wei = wei.masked_fill(tril == 0, float('-inf'))
print(wei.shape)
#dim=1 kolumna wiec kolumna jest jest broadcastowana zeby otrzymac w weirszach sume do 1
wei = F.softmax(wei,dim=1)  #exp() w softmax da kazde -inf na 0 bo tak dzial a 0 na 1 i potem zsumuje sobie wszystko do jednej kolumny i zrobi broadcasting
xbow3 = wei @x
xbow3

NameError: name 'T' is not defined

In [ ]:
-

W kontekście mechanizmu self-attention, klucze (keys) i zapytania (queries) pełnią centralną rolę w określaniu, jak każdy token (słowo lub inny element sekwencji) w sekwencji "zwraca uwagę" na inne tokeny. Oto, co każdy z nich robi i jak ze sobą współpracują:

Klucze (Keys)

    Rola: Klucze reprezentują tokeny w sekwencji w kontekście, na który mogą być zwrócone zapytania. Można je postrzegać jako etykiety lub identyfikatory, które pomagają określić, jakie informacje są dostępne dla zapytania.
    Jak działają: Każdy token w sekwencji jest transformowany do przestrzeni kluczy za pomocą warstwy liniowej (w Twoim przykładzie key = nn.Linear(C, head_size, bias=False)), co pozwala na generowanie reprezentacji każdego tokenu jako klucz.

Zapytania (Queries)

    Rola: Zapytania są reprezentacjami tokenów, które "pytają" o inne tokeny w sekwencji, aby określić, jak bardzo powinny na nie zwracać uwagę. W praktyce oznacza to ustalanie, które tokeny są najbardziej istotne dla danego tokenu.
    Jak działają: Podobnie jak klucze, każdy token jest transformowany do przestrzeni zapytań za pomocą innej warstwy liniowej (w Twoim przykładzie query = nn.Linear(C, head_size, bias=False)), generując reprezentację każdego tokenu jako zapytanie.



In [ ]:
#Version 4 self-attention
torch.manual_seed(1337)
B,T,C = 4,8,32
#tensor o rozmairze podanym i randomowe liczby z rozkladu normalnego
x = torch.randn(B,T,C)

#single head performance
head_size = 16
key = nn.Linear(C,head_size, bias = False)
query = nn.Linear(C,head_size, bias = False)
value = nn.Linear(C,head_size, bias = False)
k = key(x) # (B,T,head_size)
q = query(x) # (B,T,head_size)


#wei reprezentuje iloczyn skalarny(Dot product) zapytania dla tokena i-tego i klucza dla tokena j-tego.
#Iloczyn skalarny jest miarą podobieństwa: mówi nam, jak bardzo token reprezentowany przez zapytanie (query)
#pasuje do tokenu reprezentowanego przez klucz (key).

# wei jest przekazywane do softmax pozniej wiec gdy jedna wartosc jest znaczenie wieksza od pozostałych
# wynikowy wektor prawdopodobieństw będzie bardzo bliski wektorowi "one-hot" czyli powiekszy roznice miedzy najwikeszym i mniejszymi wartosciami.
# dltaego chcemy zaby wartosci byly ekstremalne
# wiec robi sie wei = q @ k.transpose(-2, -1) * head_size**-0.5

wei = q @ k.transpose(-2, -1) * head_size**-0.5 # (B,T,16) @ (B,16,T) --> (B,T,T)
#przy mnozeni 3 wymiarowym popatrz tylko naostanie 2 wymiary (T,16) (16,T) wiec da nam T,T ale dla batcha jest (B,T,T)
# (B,T,16) @ (B,16,T) --> (B,T,T) tutaj  to mnozenie (T,16) (16,T) odbywa sie rownolegle dla wszytskich przykladow przez Batch dimension
# pierwsze T mowi o tym dla jakiego tokenu jest generowana uwaga
# ostatnie T mowi o tym na jakich tokenach powinien sie skupić  gdy generuje uwage dla 1 Tokenu


#tworzy trojkat z 1 na przekatnych i pod spodem
tril = torch.tril(torch.ones(T,T))

#mowimy ze tokeny z przyszlosci nie moga sie komunikowac.
wei = wei.masked_fill(tril==0,float('-inf')) # kiedy jest prawdziwe w tensorze wartosc zostaje zastapiona tym co podalismy
#tutaj dzieki softmax nie ma liczb ujemnych i dystrybucja sumuje sie do 1 w kazdym wierszu
wei = F.softmax(wei,dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape


NameError: name 'torch' is not defined

In [ ]:
#Attention to mechanizm komunikacyjny

#B - batch. pierwszy wymiar cyli wiersze. To są niezależne przykłądy które nie gadają ze sobą.
#tylko w swoim przykladzie tokeny gadaja ze soba.

#Attention to takie ogolne stwierzenie, jest self-attention i cross-attention
#self-attention to kiedy keys queries i values pochodza od tego samego źródła
#cross-attention to kiedy queries z jednego a queries i values od innego zrodla

In [ ]:
wei

wytlumacz mi to jak dla debila
ChatGPT

Oczywiście, spróbujmy to wyjaśnić w prostszy sposób, krok po kroku, używając analogii:

Wyobraź sobie, że jesteś w dużym pokoju pełnym ludzi (tokenów) na przyjęciu (sekwencji). Każda osoba (każdy token) chce zdecydować, do kogo najbardziej należy zwrócić uwagę podczas rozmowy. W tym celu każdy używa trzech narzędzi: klucza, zapytania i wartości.

    Zapytanie (Query): To Twoje osobiste pytanie. Wyobraź sobie, że trzymasz kartkę, na której napisane jest "Jak bardzo powinienem zainteresować się każdą osobą tutaj?". Twoje pytanie jest unikalne dla Ciebie i może być inne niż pytania innych osób.

    Klucz (Key): Każda inna osoba na przyjęciu również trzyma swoją własną kartkę, na której jest napisane "To są moje cechy, które mogą Cię zainteresować". To jest ich klucz. Kiedy porównujesz swoje pytanie (query) z ich kluczem, możesz zdecydować, jak bardzo jesteś nimi zainteresowany.

    Wartość (Value): Oprócz ich klucza, każda osoba ma również broszurę (to jest ich wartość), która zawiera więcej informacji o nich, na przykład ich historię życia, zainteresowania itd. Jeśli zdecydujesz, że ktoś jest interesujący na podstawie porównania Twojego pytania z ich kluczem, będziesz chciał poznać więcej z ich broszury.

Teraz, jak to działa razem w self-attention:

    Obliczenie uwagi: Korzystając z Twojego pytania, porównujesz je z kluczem każdej osoby, aby określić, jak ważna dla Ciebie jest każda z nich. To jest jakbyś wykonywał "mnożenie" swoich pytań przez ich klucze, aby określić, kto jest najbardziej związany z Twoim zainteresowaniem.

    Stosowanie uwagi: Po ustaleniu, kto jest dla Ciebie najciekawszy, spojrzysz do ich broszur (wartości) i zbierasz z nich informacje. Ale nie czytasz każdej broszury w całości; zamiast tego, skupiasz się bardziej na tych osobach, które uznałeś za ważniejsze (czyli na tych, którym 'przydzieliłeś' więcej uwagi).

    Tworzenie Twojej historii: Na koniec, bierzesz wszystkie informacje, które zebrałeś od innych (korzystając z ich wartości), ważysz je według tego, jak bardzo zainteresowały Cię ich klucze (czyli ile 'uwagi' im przydzieliłeś), i tworzysz swoją historię (czyli nową reprezentację siebie), która jest zsumowaniem tego, co dowiedziałeś się od innych i jak bardzo to dla Ciebie istotne.

Podsumowując, w kontekście sieci neuronowych, "zapytanie" reprezentuje, jak ważne są dla Ciebie informacje z innych tokenów, "klucz" pokazuje, jak istotne mogą być informacje innych tokenów dla Ciebie, a "wartość" to faktyczne informacje, które chcesz zebrać od innych tokenów, bazując na tym, jak ważne są dla Ciebie według klucza i zapytania. Mechanizm uwagi (attention) pozwala każdemu tokenowi w sekwencji 'decydować', na których innych tokenach 'skupić uwagę', tworząc nową reprezentację opartą na całej dostępnej sekwencji.

In an "encoder" attention block just delete the single line that does masking with tril, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.

"Scaled" attention additional divides wei by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [ ]:
-#klasa Head zeby lepiej jakos to przedstawic

class Head(nn.Module):
  """ one head of self-attention """

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n.embd, head_size,bias = False)
    self.query = nn.Linear(n.embd, head_size,bias = False)
    self.value = nn.Linear(n.embd, head_size,bias = False)
    #To jest akurat buffer i on nie ma gradientow jak by co tne tril
    self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size))) # tu jest maskymalna ilsoc jaka przetwarzam

    self.dropout = nn.Dropout(dropout)

  def forward(self,x):
    B,T,C = x.shape

    k = self.key(x) # (B,T,C)
    q = self.query(x) # (B,T,C)
    wei = q @ k.transpose(-2,-1) * C**-0,5

    #jesli masz juz zdefiniowane w konstruktorze wo musisz tylk podac
    # odpowiednie wymiary maski kiedy rzeczywiste sekwencje, które przetwarzasz, są krótsze. [:T,:T]
    # np przetwarzasz 8 na 8 a block size to 543
    wei = wei.masked_fill(self.tril[:T,:T] ==0,float('inf'))
    wei = F.softmax(wei,dim=-1)
    wei = self.dropout(wei)
    v = self.value(x)
    out = wei @ v
    return out

In [ ]:
#klasa MultiHead
class MultiHeadAttention(nn.Module):
  """ one head of self-attention """

  def __init__(self,num_heads,head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.projections = nn.Linera(n_embd,n_embd)
    self.dropout = nn.Dropout(dropout)


#konkatonacja wynikow wszytwskich headow po wymiaerze C bo zostaly rozdzielone wczesniej
  def forward(self,x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.dropout(out)
    out = self.projections(out) # to jest powrót do sciezki rezydualnej
    return out


Rozumiem, chcesz wiedzieć, jak w praktyce działa przetwarzanie danych przez głowy uwagi (attention heads) w modelu transformatora dla wielu batchy danych.

Załóżmy, że masz tensor wejściowy o wymiarach [B, T, C], gdzie:

    B = 16 (rozmiar batcha: liczba niezależnych sekwencji przetwarzanych równolegle),
    T = 8 (długość sekwencji: liczba tokenów w każdej sekwencji),
    C = 32 (wymiar cech: rozmiar embeddingu dla każdego tokenu).

Ponadto, załóżmy, że masz n_head = 4 głowy uwagi w warstwie Multi-Head Attention.

Oto jak to działa:

    Rozdzielenie na Głowy Uwagi (Splitting into Attention Heads):
    Każda "głowa" uwagi działa na części danych wejściowych. W praktyce, wymiar C jest dzielony równo na n_head głów, więc każda głowa pracuje z częścią cech każdego tokenu. Przy n_head = 4 i C = 32, każda głowa używa 8 cech z każdego tokenu (C / n_head = 32 / 4 = 8).

    Przetwarzanie Batchy:
        Dla każdego batcha danych (B = 16 sekwencji), wszystkie sekwencje są przetwarzane równolegle przez sieć.
        Gdy mówimy o "kilku batchach", np. 30, oznacza to, że proces przetwarzania opisany poniżej jest powtarzany 30 razy, każdy raz z innym zestawem sekwencji.

    Przebieg Przez Głowy Uwagi:
        W ramach każdej głowy uwagi, dokonuje się przekształceń linearowych (za pomocą warstw nn.Linear) dla kluczy (k), zapytań (q) i wartości (v) z odpowiednich podziałów danych wejściowych. Dla każdej sekwencji w batchu, dla każdej głowy, obliczane są osobne zestawy k, q, v.
        Następnie, dla każdej głowy i każdej sekwencji, obliczany jest iloczyn skalarny między zapytaniami a kluczami, a wynik jest skalowany, maskowany (jeśli to potrzebne, by zapobiec "spojrzeniom w przyszłość"), a na koniec stosowany jest softmax do określenia wag uwagi.
        Wagi uwagi są używane do ważenia wartości (v) dla każdej głowy, co daje częściowe wyniki dla każdej głowy.

    Łączenie Wyników z Głów (Combining Results from Heads):
        Wyniki z wszystkich głów są następnie łączone (zwykle przez konkatenację wzdłuż wymiaru cech, a potem ewentualnie przekształcone przez dodatkową warstwę liniową), by zrekonstruować pełny wymiar C dla każdego tokenu.

    Dalsze Przetwarzanie:
        Wynik z warstwy uwagi wielogłowicowej może być następnie przetworzony przez pozostałe części bloku transformera, jak warstwy Feed-Forward, normalizację warstwową, itd.

Podsumowując, w praktyce, dla wielu batchy danych, cały opisany proces jest powtarzany dla każdego batcha niezależnie. Każda "głowa" uwagi przetwarza wszystkie sekwencje w każdym batchu równolegle, ale skupia się na innych "aspektach" danych dzięki operowaniu na różnych podzbiorach cech. Mechanizm wielogłowicowej uwagi pozwala modelowi na równoczesne przetwarzanie różnych rodzajów relacji między tokenami w sekwencji.

Rozumiem, chcesz zobaczyć konkretny przykład numeryczny. Załóżmy, że mamy następujące wymiary:

    Rozmiar batcha (B) = 16,
    Długość sekwencji (T) = 8,
    Całkowita liczba cech (C) = 32,
    Liczba głów uwagi (n_head) = 4,
    Liczba cech na głowę (head_size) = C / n_head = 32 / 4 = 8.

Rozpoczynając od tensora danych wejściowych o wymiarach [B, T, C], co w naszym przykładzie daje [16, 8, 32].

    Operacja w Multi-Head Attention:

    Na początku nie rozdzielamy jeszcze danych na głowy. Mamy tensor o wymiarach [16, 8, 32], co oznacza, że mamy 16 sekwencji, każda o długości 8 tokenów, i każdy token jest reprezentowany przez 32-wymiarowy wektor cech.

    Transformacja dla poszczególnych głów:

    Przygotowujemy dane dla każdej z głów uwagi. W modelach transformatorów, to zazwyczaj zachodzi wewnątrz operacji, które transformują oryginalne dane do przestrzeni kluczy (k), zapytań (q) i wartości (v) za pomocą osobnych warstw liniowych dla każdej głowy. Ale dla uproszczenia załóżmy, że już mamy nasze dane we właściwym formacie i chcemy tylko zrozumieć podział na głowy.

    Zmiana kształtu danych:

    Przekształcamy nasz tensor do nowego kształtu, który separuje głowy uwagi. To znaczy, przekształcamy z [B, T, C] do [B, n_head, T, head_size]. W naszym przypadku, to przekształcenie zmienia wymiary z [16, 8, 32] na [16, 4, 8, 8]. Oznacza to, że teraz każda z czterech głów ma do dyspozycji 8-wymiarowe reprezentacje dla każdego z 8 tokenów, i przetwarzamy 16 takich sekwencji równolegle.

    Interpretacja nowego tensora:

    W nowo przekształconym tensorze:
        Pierwszy wymiar (16) wciąż reprezentuje rozmiar batcha,
        Drugi wymiar (4) reprezentuje liczbę głów uwagi,
        Trzeci wymiar (8) reprezentuje długość sekwencji (tyle samo co T),
        Czwarty wymiar (8) reprezentuje nową liczbę cech na głowę (tyle samo co head_size).

Każda "głowa" uwagi w modelu będzie teraz operować na swoim własnym, 8-wymiarowym podprzestrzeniach danych dla każdego tokenu, i to wszystko dzieje się równolegle dla wszystkich sekwencji w batchu. Po przetworzeniu przez mechanizmy uwagi, wyniki z różnych głów mogą być na różne sposoby integrowane, na przykład przez ponowne połączenie ich w większe reprezentacje i przeprowadzenie dalszych transformacji.

In [ ]:
#Feed forward -  to jest potrdzebne zeby siec pomyslala nad zebranymi danymui z self-attention
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd), # W pracy doktorackiej jest razy 4
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),  # powrot do sciezki rezydualnej

            #Dodajesz tuz przed zlaczeniem sie polaczen rezydualnych do orginalenej sciezki
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
#Optymalizacja bardzo głebkoich sieci neuronowych:
#Są 2 sposoby:
#____PIERWSZY_____
# Skip Connections (Połączenia Pomijające) zwane także Residual Connections (Połączenia Rezydualne):
# Chodzi o to że masz dodatką droge dla gradientu bo przecież przy operacjach dodawania gradient jest po prostu przekazywany.
#______DRUGI______
#LayerNorm - layer normalization.
#inna normalziacja niz batch normalization
#W batch normalization normalizujemy po T czyli kolumnach czyli po cechach
#W layer normalizujemy po wierszach czyli B czyli batchach
#Podejscie sie zmienilo i teraz warstwa layernorm jest stosowana przed transformacja a nie po jak w pracy Attention


In [ ]:
class LayerNorm:

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(0, keepdim=True) # batch mean
    xvar = x.var(0, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta

    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

In [ ]:
#Droppout - latwy sposob zeby zapobiec overfitingowi
#za kazdym razem jak jest forward pass randomowo wybiera i wylacza jakies neurony
#Jest to technika regularyzacji
#np dropout = 0.2 kazdy forward pass 20 procent kalkulacji sa wywalane i zrzucane do 0.